In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chatgpt-users-reviews/ChatGPT_Reviews.csv


In [2]:
# #This project works by doing the following:
# 1.EDA cleaning and Language Translation using Google API
# 2. Sentiment Classifier using NLTK
# 3. Topic categorization, identifying users most frequent words used 
# in their reviews to drive business insights
# 4. Finally A ML model that predicts Customer sentiment that can be used
# to identify positivie, negative and neutral feedback 

# #The use of dataset CHATGPT_Reviews is attributed to the author Anand Shaw

Importing all neccesary libraries

In [3]:
# These might need to be installed
!pip install google-cloud-translate
!pip install bertopic
!pip install vaderSentiment


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.0 MB/s eta 0:00:00


In [4]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("ggplot")

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')



[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
!ls /kaggle/input


chatgpt-users-reviews


In [6]:
#import dataset.
#Get the dataset From Anand Profile ChatGPT_Users_Review
df = pd.read_csv("/kaggle/input/chatgpt-users-reviews/ChatGPT_Reviews.csv")
df.head(5)

,Review Id,Review,Ratings,Review Date
0,6fb93778-651a-4ad1-b5ed-67dd0bd35aac,good,5,8/23/2024 19:30
1,81caeefd-3a28-4601-a898-72897ac906f5,good,5,8/23/2024 19:28
2,452af49e-1d8b-4b68-b1ac-a94c64cb1dd5,nice app,5,8/23/2024 19:22
3,372a4096-ee6a-4b94-b046-cef0b646c965,"nice, ig",5,8/23/2024 19:20
4,b0d66a4b-9bde-4b7c-8b11-66ed6ccdd7da,"this is a great app, the bot is so accurate to...",5,8/23/2024 19:20


In [7]:
#checking for null
df.isnull().sum()

Review Id      0
Review         6
Ratings        0
Review Date    0
dtype: int64

In [8]:
# Drop rows where the 'Review' column has null values
df = df.dropna(subset=['Review'])

# Verify the null values are dropped
print(df.isnull().sum())


Review Id      0
Review         0
Ratings        0
Review Date    0
dtype: int64


In [9]:
#check for duplicates
# Identify columns with duplicates
for column in df.columns:
    if df[column].duplicated().sum() > 0:
        print(f"Column '{column}' has {df[column].duplicated().sum()} duplicate values.")


Column 'Review Id' has 3573 duplicate values.
Column 'Review' has 71230 duplicate values.
Column 'Ratings' has 196716 duplicate values.
Column 'Review Date' has 48528 duplicate values.


Review Id should be unique identifier and hence not have duplicates, in this case investigate more to know why so many

In [10]:
# Check for duplicate Review_Id values
duplicates = df[df['Review Id'].duplicated()]
print(duplicates)


                                   Review Id  \
3208    02229a18-6bb2-40c4-8302-b0c430505f65   
4896    52e3fd17-883b-4cdb-95fe-303fa50d285d   
5476    1ece904e-2a2d-4ced-b819-d08239dd24ac   
5547    86e87741-70d7-4f3b-aef7-446179b204fc   
5948    4299411b-fb16-4785-970f-b57ebb484857   
...                                      ...   
195802  c0bf774c-1c46-4180-875b-70dd231d78e7   
195980  cf6da41f-2b70-4376-91b4-ab3c789b8ddf   
196045  873bf35d-7dd5-4cce-a250-83a2512f7744   
196163  b2b44c0f-e4f7-40e4-adca-1b375acbc788   
196498  8bb7d91b-de5a-4b6a-9c6f-de4b064bee4d   

                                                   Review  Ratings  \
3208    Chat gpt is not allowing me to login in websit...        1   
4896                         my favourite apps.......🥹🫶🏻💗        5   
5476    For me ,it's so useful.what ever u ask ,it wil...        5   
5547                             Very good app keep it up        4   
5948        There seems to be a cap on detailed responses        5   
...